In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.readStream.format('socket')\
                        .option('host','localhost')\
                        .option('port',12345)\
                        .load()

22/06/14 09:38:51 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
writer = df.writeStream.outputMode('append')\
                        .format('console')

In [5]:
query = writer.start()

22/06/14 09:40:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bcc84238-e4e8-41da-b922-36cfe915f940. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 09:40:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|   Hi|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---------------+
|          value|
+---------------+
|Hello everybody|
+---------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+
|  value|
+-------+
|PysPark|
+-------+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|ksdjlkjflkjsfljsk...|
+--------------------+



In [6]:
query.stop()

In [7]:
df_stream = spark.readStream.format('socket')\
.option('host','localhost')\
.option('port',12345)\
.load()

22/06/14 10:47:07 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [8]:
df_stream.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
from pyspark.sql.functions import split, explode

In [10]:
lines_splitted = df_stream.select(split(df_stream['value'],' ').alias('splittedlines'))

In [11]:
lines_splitted

DataFrame[splittedlines: array<string>]

In [12]:
lines_splitted.printSchema()

root
 |-- splittedlines: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
words = lines_splitted.select(explode(lines_splitted['splittedlines']).alias('word'))

In [14]:
words.printSchema()

root
 |-- word: string (nullable = true)



In [15]:
wordcounst = words.groupBy('word').count()

In [16]:
wordcounst.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [20]:
words_new = df_stream.select(explode(split(df_stream['value'],' ')).alias('word'))\
.groupBy('word').count()

In [28]:
writer = words_new.writeStream.format('console').outputMode('complete')\
.trigger(processingTime='1 second')\
.option('checkpointLocation',checkpointDir)

In [29]:
writer_0 = df_stream.writeStream.format('console').outputMode('append').trigger(processingTime='1 second')

In [30]:
writer_1 = lines_splitted.writeStream.format('console').outputMode('append').trigger(processingTime='1 second')

In [31]:
writer_2 = words.writeStream.format('console').outputMode('append').trigger(processingTime='1 second')

In [32]:
writer_3 = wordcounst.writeStream.format('console')\
.outputMode('complete')\
.trigger(processingTime='1 second')\
.option('checkpointLocation',checkpointDir3)

In [26]:
checkpointDir = 'chkpnt'

In [27]:
checkpointDir3 = 'chkpnt3'

In [33]:
query = writer.start()

22/06/14 11:12:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



22/06/14 11:13:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 61934 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    1|
|  how|    1|
|   Hi|    4|
|Hello|    3|
| you?|    1|
|  are|    1|
+-----+-----+



22/06/14 11:14:28 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 29789 milliseconds
22/06/14 11:14:52 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 24380 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    1|
|  how|    1|
|   Hi|    5|
|Hello|    3|
| you?|    2|
|  are|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    1|
|  how|    1|
|   Hi|    6|
|Hello|    3|
| you?|    2|
|  are|    1|
+-----+-----+



22/06/14 11:16:43 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 29167 milliseconds


In [34]:
query.stop()

In [35]:
query_1 = writer_1.start()

22/06/14 11:19:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-29fc1243-e837-4b86-85c6-fa0c224b15a5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 11:19:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------+
|splittedlines|
+-------------+
+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|       splittedlines|
+--------------------+
|[Hi, how, are, yo...|
+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|       splittedlines|
+--------------------+
|[Hi, Hello, Hi, H...|
+--------------------+



In [36]:
query_1.stop()

In [37]:
query_2 = writer_2.start()

22/06/14 11:20:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-feb98492-5731-4f2a-8c19-af0ab4928046. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 11:20:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|word|
+----+
+----+



22/06/14 11:20:30 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1108 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----+
|word|
+----+
|  Hi|
| how|
| are|
| you|
|   ?|
+----+



In [38]:
query_2.stop()

In [41]:
from pyspark.sql.functions import *
lines = (spark
         .readStream.format("socket")
         .option("host", "localhost")
         .option("port", 12345)
         .load())
#words = lines.select(split(col("value"), "\\s").alias("word"))
words = lines.select(explode(split(lines.value, " ")).alias("word"))
counts = words.groupBy("word").count()
checkpointDir = "chkpnt"
streamingQuery = (counts.writeStream
                  .format("console")
                  .outputMode("complete")
                  .trigger(processingTime="30 second")
                  .option("checkpointLocation", 'chk')
                  .start())

22/06/14 11:22:27 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
22/06/14 11:22:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



22/06/14 11:23:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 30000 milliseconds, but spent 38921 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|   Hi|    2|
|Hello|    2|
+-----+-----+



In [42]:
streamingQuery.stop()

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.readStream.format('text').load('MyInputStream/')

In [3]:
writer = df.writeStream.outputMode('append')\
.format('console')\
.option('truncate',False)\
.option('numRows',10)

In [5]:
query = writer.start()

22/06/14 11:43:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ed5f4d24-cf01-4fb1-8a9f-c50ca5e48017. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 11:43:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------------------+
|value                                 |
+--------------------------------------+
|date,delay,distance,origin,destination|
|1011245,6,602,ABE,ATL                 |
|1020600,-8,369,ABE,DTW                |
|1021245,-2,602,ABE,ATL                |
|1020605,-4,602,ABE,ATL                |
|1031245,-4,602,ABE,ATL                |
|1030605,0,602,ABE,ATL                 |
|1041243,10,602,ABE,ATL                |
|1040605,28,602,ABE,ATL                |
|1051245,88,602,ABE,ATL                |
+--------------------------------------+
only showing top 10 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------------------------+
|value                                 |
+--------------------------------------+
|date,delay,distance,origin,destination|
|1080600,0,369,ABE,DTW                 |
|

In [6]:
query.stop()

In [13]:
from pyspark.sql.types import (StructType, StructField,
                               StringType, IntegerType)

recordSchema = StructType([StructField('date', StringType(), True),
                           StructField('delay', IntegerType(), True),
                           StructField('distance', IntegerType(), True),
                           StructField('origin', StringType(), True),
                           StructField('destination', StringType(), True)])

In [20]:
recordSchema2 = StructType([StructField('date', StringType(), True),
                           StructField('delay', IntegerType(), True),
                           StructField('distance', IntegerType(), True),
                           ])

In [8]:
df = spark.readStream.format('csv')\
.schema(recordSchema)\
.load('MyInputStream/')

In [16]:
df = spark.readStream.format('parquet')\
.schema(recordSchema)\
.load('MyInputStream/')

In [21]:
df = spark.readStream.format('parquet')\
.schema(recordSchema2)\
.load('MyInputStream/')

In [22]:
writer = df.writeStream.outputMode('append')\
.format('console')\
.option('truncate',False)\
.option('numRows',10)

In [23]:
query = writer.start()

22/06/14 11:56:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c67d5288-9974-408e-bf97-46772a8d52df. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 11:56:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----+--------+
|date   |delay|distance|
+-------+-----+--------+
|1080600|0    |369     |
|1081230|33   |369     |
|1080625|1    |602     |
|1080607|5    |569     |
|1081219|54   |569     |
|1091215|43   |602     |
|1090600|151  |369     |
|1091725|0    |602     |
|1091230|-4   |369     |
|1090625|8    |602     |
+-------+-----+--------+
only showing top 10 rows



In [24]:
query.stop()

In [31]:
df = spark.readStream.format('socket')\
.option('host','localhost')\
.option('port',12345)\
.load()

22/06/14 12:56:40 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [32]:
writer = df.writeStream.outputMode('append')\
.format('text')\
.option('path','OutStream/')\
.option('checkpointLocation','chkpnt')

In [33]:
query= writer.start()

22/06/14 13:00:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
df_mytable.show()

+-------------------+-----+
|               word|count|
+-------------------+-----+
|      [Hello, Hi, ]|    1|
|[Hi, how, are, you]|    1|
+-------------------+-----+



In [9]:
streamingQuery.stop()

In [14]:
df = spark.readStream.format('csv')\
.schema(recordSchema)\
.load('MyInputStream/')

In [15]:
writer = df.writeStream.outputMode("append") \
    .format("memory")  \
    .queryName('myTable')

In [16]:
query= writer.start()

22/06/14 13:32:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9ad0dad3-7415-4ab0-897f-27a7ac67b552. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 13:32:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [17]:
spark.sql('SELECT * FROM myTable').show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|   date| null|    null|origin|destination|
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE| 

In [18]:
query.stop()

In [20]:
df2 = df.groupBy('origin').count()

In [21]:
writer = df2.writeStream.outputMode("complete") \
    .format("memory")  \
    .queryName('myTable')

In [23]:
query= writer.start()

22/06/14 13:34:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ccba14f3-a0ad-40d5-94fa-e89d94cabe41. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/14 13:34:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [24]:
spark.sql('SELECT * FROM myTable').show()

+------+-----+
|origin|count|
+------+-----+
+------+-----+



In [25]:
spark.sql('SELECT * FROM myTable').show()

+------+-----+
|origin|count|
+------+-----+
|   ABE|   19|
|origin|    1|
+------+-----+



In [26]:
spark.sql('SELECT * FROM myTable').show()

+------+-----+
|origin|count|
+------+-----+
|   ABE|   19|
|origin|    1|
+------+-----+

